In [ ]:
import pathlib
import re

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Загрузка данных

In [ ]:
DATA_DIR = pathlib.Path("../data")

In [ ]:
train_df = pd.read_csv(DATA_DIR.joinpath("train.csv"), index_col="pair_id")
test_df = pd.read_csv(DATA_DIR.joinpath("test.csv"), index_col="pair_id")

In [ ]:
train = train_df.copy()
test = test_df.copy()

In [ ]:
companies = pd.read_csv(DATA_DIR.joinpath('companies_sorted.csv'))

In [ ]:
companies = companies[(companies['name'].notna()) & (companies['industry'].notna())]

# Очистка данных

In [ ]:
train["name_1"] = train["name_1"].str.lower()
train["name_2"] = train["name_2"].str.lower()

test["name_1"] = test["name_1"].str.lower()
test["name_2"] = test["name_2"].str.lower()

In [ ]:
companies['name'] = companies['name'].str.lower()

In [ ]:
legal_entities = ["ltd\.", "co\.", "inc\.", "b\.v\.", "s\.c\.r\.l\.", "gmbh", "pvt\.", "llc", "corp", "corp\.",
                  "bv", "s\.a\.", "c\.v\.", "ltda", "cv", "sa", "ca", "c\.a\."]
legal_re = re.compile(r'\s*\b(?:' + '|'.join([rf"{entity}" for entity in legal_entities]) + r')(?!\S)')

In [ ]:
for dataset in (train, test):
    dataset.replace(to_replace=legal_re, value="", inplace=True, regex=True)

In [ ]:
for dataset in (train, test):
    dataset.replace(to_replace=re.compile(r"[^\w\s]"), value=" ", inplace=True, regex=True)
    dataset.replace(to_replace=re.compile(r"\s+"), value=" ", inplace=True, regex=True)

In [ ]:
companies.replace(to_replace=legal_re, value="", inplace=True, regex=True)
companies.replace(to_replace=re.compile(r"[^\w\s]"), value=" ", inplace=True, regex=True)
companies.replace(to_replace=re.compile(r"\s+"), value=" ", inplace=True, regex=True)

In [ ]:
companies.drop_duplicates(subset=['name'], inplace=True)

# Сопоставление названий

### Полное сопоставление

In [ ]:
cols_to_merge = ['name', 'industry', 'current employee estimate', 'total employee estimate', 'year founded', 'locality']

In [ ]:
train = train.merge(companies[cols_to_merge], left_on='name_1', right_on='name', how='left')
train.drop(columns=['name'], inplace=True)
train.rename(columns={col: col + '_n1' for col in cols_to_merge}, inplace=True)

test = test.merge(companies[cols_to_merge], left_on='name_1', right_on='name', how='left')
test.drop(columns=['name'], inplace=True)
test.rename(columns={col: col + '_n1' for col in cols_to_merge}, inplace=True)

In [ ]:
train = train.merge(companies[cols_to_merge], left_on='name_2', right_on='name', how='left')
train.drop(columns=['name'], inplace=True)
train.rename(columns={col: col + '_n2' for col in cols_to_merge}, inplace=True)

test = test.merge(companies[cols_to_merge], left_on='name_2', right_on='name', how='left')
test.drop(columns=['name'], inplace=True)
test.rename(columns={col: col + '_n2' for col in cols_to_merge}, inplace=True)

### Приближенное сопоставление

In [ ]:
fuzzy_cols_to_merge = ['first_w', 'industry', 'current employee estimate', 'total employee estimate']

In [ ]:
companies['first_w'] = companies['name'].apply(lambda x: x.split()[0] if len(x.split()) else 'None')

In [ ]:
trunc_companies = companies.drop_duplicates(subset=['first_w'])

In [ ]:
train['first_w1'] = train['name_1'].apply(lambda x: x.split()[0] if len(x.split()) else 'None')
test['first_w1'] = test['name_1'].apply(lambda x: x.split()[0] if len(x.split()) else 'None')

train['first_w2'] = train['name_2'].apply(lambda x: x.split()[0] if len(x.split()) else 'None')
test['first_w2'] = test['name_2'].apply(lambda x: x.split()[0] if len(x.split()) else 'None')

In [ ]:
first_w_name1_train = pd.DataFrame(train['first_w1']).merge(trunc_companies[fuzzy_cols_to_merge], left_on='first_w1', right_on='first_w', how='left')
first_w_name1_test = pd.DataFrame(test['first_w1']).merge(trunc_companies[fuzzy_cols_to_merge], left_on='first_w1', right_on='first_w', how='left')

first_w_name2_train = pd.DataFrame(train['first_w2']).merge(trunc_companies[fuzzy_cols_to_merge], left_on='first_w2', right_on='first_w', how='left')
first_w_name2_test = pd.DataFrame(test['first_w2']).merge(trunc_companies[fuzzy_cols_to_merge], left_on='first_w2', right_on='first_w', how='left')

In [ ]:
train['industry_n1'].fillna(first_w_name1_train['industry'], inplace=True)
train['industry_n2'].fillna(first_w_name2_train['industry'], inplace=True)

train['current employee estimate_n1'].fillna(first_w_name1_train['current employee estimate'], inplace=True)
train['current employee estimate_n2'].fillna(first_w_name2_train['current employee estimate'], inplace=True)

train['total employee estimate_n1'].fillna(first_w_name1_train['total employee estimate'], inplace=True)
train['total employee estimate_n2'].fillna(first_w_name2_train['total employee estimate'], inplace=True)

In [ ]:
test['industry_n1'].fillna(first_w_name1_test['industry'], inplace=True)
test['industry_n2'].fillna(first_w_name2_test['industry'], inplace=True)

test['current employee estimate_n1'].fillna(first_w_name1_test['current employee estimate'], inplace=True)
test['current employee estimate_n2'].fillna(first_w_name2_test['current employee estimate'], inplace=True)

test['total employee estimate_n1'].fillna(first_w_name1_test['total employee estimate'], inplace=True)
test['total employee estimate_n2'].fillna(first_w_name2_test['total employee estimate'], inplace=True)

In [ ]:
train['industry_n1'].fillna('None', inplace=True)
train['industry_n2'].fillna('None', inplace=True)

train['current employee estimate_n1'].fillna(train['current employee estimate_n1'].mean(), inplace=True)
train['current employee estimate_n2'].fillna(train['current employee estimate_n2'].mean(), inplace=True)

train['total employee estimate_n1'].fillna(train['total employee estimate_n1'].mean(), inplace=True)
train['total employee estimate_n2'].fillna(train['total employee estimate_n2'].mean(), inplace=True)

In [ ]:
test['industry_n1'].fillna('None', inplace=True)
test['industry_n2'].fillna('None', inplace=True)

test['current employee estimate_n1'].fillna(test['current employee estimate_n1'].mean(), inplace=True)
test['current employee estimate_n2'].fillna(test['current employee estimate_n2'].mean(), inplace=True)

test['total employee estimate_n1'].fillna(test['total employee estimate_n1'].mean(), inplace=True)
test['total employee estimate_n2'].fillna(test['total employee estimate_n2'].mean(), inplace=True)

In [ ]:
train.head()

In [ ]:
cols_to_save = ['industry_n1', 'current employee estimate_n1', 'total employee estimate_n1',
                'industry_n2', 'current employee estimate_n2', 'total employee estimate_n2']

In [ ]:
train.index = train_df.index
test.index = test_df.index

In [ ]:
train[cols_to_save].to_csv(DATA_DIR.joinpath('train_external.csv'))

In [ ]:
test[cols_to_save].to_csv(DATA_DIR.joinpath('test_external.csv'))